In [1]:
# Abiodun Ajadi - INSY 7420 Homework 1, Problem 5
import csv
import gurobipy as gp
from gurobipy import Model, Env, GRB

def crude_oil_product():
    # update with options for your academic web license
    options = {
        "WLSACCESSID": "a1d427b3-ac63-4214-b598-bb01d2fc2407",
        "WLSSECRET": "c176d5f7-65d7-4715-acda-fc55b2def838",
        "LICENSEID": 2765843
    }
    
    with Env(params=options) as env, Model(env=env) as m:
        m.setParam("outputFlag", 0)
        
        # Decision variables
        y1 = m.addVar(lb=0, ub=10000, name="crude_oil_type_1")
        y2 = m.addVar(lb=0, ub=10000, name="crude_oil_type_2")
        d1 = m.addVar(lb=0, name="crude_oil_type_1_distilled")
        d2 = m.addVar(lb=0, name="crude_oil_type_2_distilled")
        c1 = m.addVar(lb=0, name="crude_oil_type_1_distilled_cracked")
        c2 = m.addVar(lb=0, name="crude_oil_type_2_distilled_cracked")
        x1 = m.addVar(lb=3000, name="heating_oil")
        x2 = m.addVar(lb=3000, name="gasoline")
        x3 = m.addVar(lb=3000, name="jetfuel")
        
        # Barrels of naphtha allocated to the products
        n_x2 = m.addVar(lb=0, name="naphtha_for_gasoline")
        n_x3 = m.addVar(lb=0, name="naphtha_for_jetfuel")
        
        # Barrels of cracked oil allocated to the products
        c1_x2 = m.addVar(lb=0, name="cracked_oil_1_for_gasoline")
        c1_x3 = m.addVar(lb=0, name="cracked_oil_1_for_jetfuel")
        c2_x3 = m.addVar(lb=0, name="cracked_oil_2_for_jetfuel")
        
        # Objective function is to maximize profit = revenue - cost
        profit = (14*x1 + 18*x2 + 16*x3) - (10*y1 + 12*y2 + 0.1*(d1 + d2) + 0.15*(c1 + c2))
        m.setObjective(profit, GRB.MAXIMIZE)
        
        # Constraints for the problem
        m.addConstr(d1 + d2 <= 15000, "distillation_capacity")
        m.addConstr(c1 + c2 <= 5000, "cracking_capacity")
        m.addConstr(d1 <= y1, "distill_crude1")
        m.addConstr(d2 <= y2, "distill_crude2")
        m.addConstr(c1 <= 0.3*d1 + 0.2*d2, "crack1_yield")
        m.addConstr(c2 <= 0.1*d1 + 0.4*d2, "crack2_yield")
        
        # Balance constraints
        m.addConstr(n_x2 + n_x3 == 0.6*d1 + 0.4*d2, "naphtha_balance")
        m.addConstr(c1_x2 + c1_x3 == 0.8*c1 + 0.7*c2, "cracked1_balance")
        m.addConstr(c2_x3 == 0.2*c1 + 0.3*c2, "cracked2_balance")
        m.addConstr(x1 == 0.4*d1 + 0.6*d2 - c1 - c2, "heating_oil_balance")
        m.addConstr(x2 == n_x2 + c1_x2, "gasoline_balance")
        m.addConstr(x3 == n_x3 + c1_x3 + c2_x3, "jetfuel_balance")
        
        # Quality constraints
        m.addConstr(0.8*c1 >= n_x2, "gasoline_quality")
        m.addConstr(d2 + 5*c1 >= d1 + 5*c2, "heating_oil_quality")
        m.addConstr(n_x3 + 1.4*c1 - 0.7*c2 >= 0, "jetfuel_quality")
        
        # Minimum production constraints
        m.addConstr(x1 >= 3000, "min_heating_oil")
        m.addConstr(x2 >= 3000, "min_gasoline")
        m.addConstr(x3 >= 3000, "min_jetfuel")
        
        m.optimize()
        
        if m.status == GRB.OPTIMAL:
            return round(m.objVal, 2)
        else:
            print(f"Optimization status: {m.status}")
            return None

print(f"The maximum profit from the production is ${crude_oil_product()}")

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2765843
Academic license 2765843 - for non-commercial use only - registered to as___@auburn.edu
The maximum profit from the production is $84700.0
